# Task 2
I will include some of the steps in the task 2 to start from the proper dataset

In [13]:
#import statements
import pandas as pd
import re
import spacy
nlp = spacy.load('en_core_web_sm')
from nltk.corpus import stopwords
import numpy as np

In [14]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [15]:
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 12.8 MB 6.8 MB/s            
You should consider upgrading via the '/Users/valedipalo/PycharmProjects/PCLAB3/venv/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [16]:
data = pd.read_csv('Data/Data_PCLab3_Twitter_Stock_Sentiment.csv')

In [17]:
data = word_count_col(data,'Text','pre_word_count')

NameError: name 'word_count_col' is not defined

In [ ]:
documents = data["Text"].tolist()

In [ ]:
nltk.download('stopwords')
%time clean_des = [preprocess(string) for string in documents[:]]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


CPU times: user 43.1 s, sys: 387 ms, total: 43.5 s
Wall time: 53.3 s


In [ ]:
data['clean_text'] = clean_des
data = word_count_col(data,'clean_text','post_word_count')

In [ ]:
data


,Text,Sentiment,pre_word_count,clean_text,post_word_count
0,Kickers on my watchlist XIDE TIT SOQ PNK CPW B...,1,20,kicker watchlist xide tit soq pnk cpw bpz aj t...,15
1,user: AAP MOVIE. 55% return for the FEA/GEED i...,1,16,aap movie return fea geed indicator trade year...,9
2,user I'd be afraid to short AMZN - they are lo...,1,18,afraid short amzn look like near monopoly eboo...,10
3,MNTA Over 12.00,1,3,mnta over,2
4,OI Over 21.37,1,3,oi over,2
...,...,...,...,...,...
5786,Industry body CII said #discoms are likely to ...,0,19,industry body cii say discom likely suffer net...,13
5787,"#Gold prices slip below Rs 46,000 as #investor...",0,15,gold price slip below rs investor book profit ...,13
5788,Workers at Bajaj Auto have agreed to a 10% wag...,1,24,worker bajaj auto agree wage cut period april ...,11
5789,"#Sharemarket LIVE: Sensex off day’s high, up 6...",1,18,sharemarket live sensex day high point nifty t...,13


# Task 3


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, RNN, SimpleRNN
from sklearn.model_selection import train_test_split 


In [ ]:
clean_text = data.clean_text.to_list()

In [ ]:
#this cell is not useful for tokenizing and padding, but just to evaluate how many tokens we have before to proceed
#takes time to run; the result is 49483

#tokens = [[token.text for token in nlp(sentence)] for sentence in clean_text]
#count = 0
#for l in tokens:
#  for w in l:
#    count += 1 
#print(count)

49483


In [ ]:
# TOKENIZING: First trial, get a strange result as i am not able to include more words reducing the num_words which should represent the minimum frequency for a word to be included 
# tokenizer = Tokenizer(num_words=100000000, lower= 1, oov_token="<OOV>")
# tokenizer.fit_on_texts(clean_text)
# word_index = tokenizer.word_index
# print(len(word_index))

6841


In [ ]:
# TOKENIZING 
tk = Tokenizer()
tk.fit_on_texts(clean_text)

tk.num_words = 1000
word_index = tk.word_index
#print(word_index)

In [ ]:
# SEQUENCES 
sequences = tk.texts_to_sequences(clean_text)
print(clean_text)
print(word_index)
print(sequences)

['kicker watchlist xide tit soq pnk cpw bpz aj trade method method see prev post', 'aap movie return fea geed indicator trade year awesome', 'afraid short amzn look like near monopoly ebook infrastructure service', 'mnta over', 'oi over', 'pgnx over', 'aap current downtrend break otherwise short term correction me term downtrend', 'monday relative weakness nyx win tie tap ice int bmc aon chk biib', 'goog ower trend line channel test volume support', 'aap watch tomorrow ong entry', 'assume fcx open tomorrow above trigger buy still much like setup', 'really worry everyone expect market rally usually exact opposite happen every time shall see soon bac spx jpm', 'aap gamco arry haverty apple extremely cheap great video', 'maykiljil post agree msft go high possibly north', 'momentum come back etfc break resistance solid volume friday ong set', 'ha hit mean resume target level', 'gameplan shot today like trend break may break oc weekly trend break back july', 'fcx gapping well above ideal en

In [ ]:
# PADDING 
padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(sequences, padding="post")
print(padded_inputs)
X = padded_inputs

[[290  33  25 ...   0   0   0]
 [  1 378 512 ...   0   0   0]
 [  2  68  10 ...   0   0   0]
 ...
 [696 617 176 ...   0   0   0]
 [ 45   4   3 ...   0   0   0]
 [ 45  49 625 ...   0   0   0]]


### LSTM 

In [ ]:
model_LSTM = Sequential()
model_LSTM.add(Embedding(500, 120, input_length = X.shape[1]))
model_LSTM.add(SpatialDropout1D(0.4))
model_LSTM.add(LSTM(176, dropout=0.2, recurrent_dropout=0.2))
model_LSTM.add(Dense(2,activation='softmax'))
model_LSTM.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
print(model_LSTM.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 18, 120)           60000     
                                                                 
 spatial_dropout1d (SpatialD  (None, 18, 120)          0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 176)               209088    
                                                                 
 dense (Dense)               (None, 2)                 354       
                                                                 
Total params: 269,442
Trainable params: 269,442
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
y = pd.get_dummies(data['Sentiment'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.1, random_state = 42)

In [ ]:
batch_size=32
model_LSTM.fit(X_train, y_train, epochs = 10, batch_size=batch_size, verbose = 'auto')

Epoch 1/10
163/163 [==============================] - 22s 101ms/step - loss: 0.5769 - accuracy: 0.6941
Epoch 2/10
163/163 [==============================] - 23s 143ms/step - loss: 0.4720 - accuracy: 0.7805
Epoch 3/10
163/163 [==============================] - 16s 100ms/step - loss: 0.4424 - accuracy: 0.7941
Epoch 4/10
163/163 [==============================] - 17s 107ms/step - loss: 0.4451 - accuracy: 0.7987
Epoch 5/10
163/163 [==============================] - 21s 131ms/step - loss: 0.4307 - accuracy: 0.8045
Epoch 6/10
163/163 [==============================] - 21s 127ms/step - loss: 0.4258 - accuracy: 0.8060
Epoch 7/10
163/163 [==============================] - 15s 93ms/step - loss: 0.4144 - accuracy: 0.8100
Epoch 8/10
163/163 [==============================] - 15s 90ms/step - loss: 0.4053 - accuracy: 0.8135
Epoch 9/10
163/163 [==============================] - 15s 90ms/step - loss: 0.3922 - accuracy: 0.8231
Epoch 10/10
163/163 [==============================] - 14s 87ms/step - loss:

In [ ]:
loss, accuracy = model_LSTM.evaluate(X_test,y_test)
print(accuracy)

19/19 [==============================] - 1s 9ms/step - loss: 0.5177 - accuracy: 0.7724
0.7724137902259827


In [ ]:
# Confusion Matrix 
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
import matplotlib.pyplot as plt

In [ ]:
y_test = y_test.iloc[:,1]
y_pred = model_LSTM.predict(X_test)
confusion_matrix(y_test, y_pred)

IndexingError: ignored

### Bi-LSTM 

In [ ]:
from keras.layers import Bidirectional

In [ ]:
model_BI = Sequential()
model_BI.add(Embedding(500, 120, input_length = X.shape[1]))
model_BI.add(SpatialDropout1D(0.4))
model_BI.add(Bidirectional(LSTM(176, dropout=0.2, recurrent_dropout=0.2)))
model_BI.add(Dense(2,activation='softmax'))
model_BI.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
print(model_BI.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 18, 120)           60000     
                                                                 
 spatial_dropout1d_3 (Spatia  (None, 18, 120)          0         
 lDropout1D)                                                     
                                                                 
 bidirectional (Bidirectiona  (None, 352)              418176    
 l)                                                              
                                                                 
 dense_1 (Dense)             (None, 2)                 706       
                                                                 
Total params: 478,882
Trainable params: 478,882
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
batch_size=32
model_BI.fit(X_train, y_train, epochs = 10, batch_size=batch_size, verbose = 'auto')

Epoch 1/10
163/163 [==============================] - 33s 171ms/step - loss: 0.5771 - accuracy: 0.6878
Epoch 2/10
163/163 [==============================] - 30s 183ms/step - loss: 0.4609 - accuracy: 0.7830
Epoch 3/10
163/163 [==============================] - 28s 169ms/step - loss: 0.4368 - accuracy: 0.7962
Epoch 4/10
163/163 [==============================] - 27s 167ms/step - loss: 0.4242 - accuracy: 0.8016
Epoch 5/10
163/163 [==============================] - 28s 169ms/step - loss: 0.4038 - accuracy: 0.8146
Epoch 6/10
163/163 [==============================] - 28s 170ms/step - loss: 0.3954 - accuracy: 0.8223
Epoch 7/10
163/163 [==============================] - 27s 163ms/step - loss: 0.3814 - accuracy: 0.8252
Epoch 8/10
163/163 [==============================] - 27s 168ms/step - loss: 0.3680 - accuracy: 0.8371
Epoch 9/10
163/163 [==============================] - 29s 175ms/step - loss: 0.3519 - accuracy: 0.8440
Epoch 10/10
163/163 [==============================] - 28s 170ms/step - l

In [ ]:
loss, accuracy = model_BI.evaluate(X_test,y_test)
print(accuracy)

In [ ]:
y_pred = model_BI.predict(X_test)

NameError: ignored

In [ ]:
y_test = y_test.iloc[:,1]
y_test=np.argmax(y_test, axis=1)
confusion_matrix(y_test, y_pred)



---



In [ ]:
model_SimpleRNN = Sequential()
model_SimpleRNN.add(Embedding(500, 120, input_length = X.shape[1]))
model_SimpleRNN.add(SpatialDropout1D(0.4))
model_SimpleRNN.add(SimpleRNN(176))
model_SimpleRNN.add(Dense(2,activation='softmax'))
model_SimpleRNN.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
print(model_SimpleRNN.summary())

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, 18, 120)           60000     
                                                                 
 spatial_dropout1d_9 (Spatia  (None, 18, 120)          0         
 lDropout1D)                                                     
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 176)               52272     
                                                                 
 dense_4 (Dense)             (None, 2)                 354       
                                                                 
Total params: 112,626
Trainable params: 112,626
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
batch_size=32
model_SimpleRNN.fit(X_train, y_train, epochs = 10, batch_size=batch_size, verbose = 'auto')

Epoch 1/10
163/163 [==============================] - 4s 20ms/step - loss: 0.5982 - accuracy: 0.6797
Epoch 2/10
163/163 [==============================] - 3s 20ms/step - loss: 0.5076 - accuracy: 0.7594
Epoch 3/10
163/163 [==============================] - 3s 19ms/step - loss: 0.4591 - accuracy: 0.7920
Epoch 4/10
163/163 [==============================] - 3s 19ms/step - loss: 0.4127 - accuracy: 0.8177
Epoch 5/10
163/163 [==============================] - 3s 19ms/step - loss: 0.3910 - accuracy: 0.8265
Epoch 6/10
163/163 [==============================] - 4s 26ms/step - loss: 0.3563 - accuracy: 0.8511
Epoch 7/10
163/163 [==============================] - 3s 20ms/step - loss: 0.3481 - accuracy: 0.8526
Epoch 8/10
163/163 [==============================] - 3s 20ms/step - loss: 0.3362 - accuracy: 0.8568
Epoch 9/10
163/163 [==============================] - 3s 20ms/step - loss: 0.3118 - accuracy: 0.8695
Epoch 10/10
163/163 [==============================] - 3s 19ms/step - loss: 0.3048 - accura

In [ ]:
model_SimpleRNN.evaluate(X_test,y_test)

19/19 [==============================] - 0s 4ms/step - loss: 0.6706 - accuracy: 0.7431


[0.6705605983734131, 0.7431034445762634]

In [ ]:
y_pred = model_SimpleRNN.predict(X_test)

NameError: ignored

In [ ]:
y_pred=np.argmax(y_pred, axis=1)


array([1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1,
       0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,
       1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1,
       1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0,
       1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0,
       1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0,

In [ ]:
loss, accuracy = model_SimpleRNN.evaluate(X_test,y_test)
print(accuracy)

In [ ]:
y_test = y_test.iloc[:,1]
y_test=np.argmax(y_test, axis=1)
confusion_matrix(y_test, y_pred)

array([[129,  83],
       [ 66, 302]])